In [1]:
from openqaoa.backends.simulators.qaoa_me_sim import QAOAMEBackendSimulator
from openqaoa.backends.simulators.qaoa_mc_sim import QAOAMCBackendSimulator
from openqaoa.backends.simulators.qaoa_qiskit_sim import QAOAQiskitBackendShotBasedSimulator
from openqaoa.qaoa_parameters import PauliOp, Hamiltonian, QAOACircuitParams, create_qaoa_variational_params, QAOAVariationalStandardParams
from openqaoa.qaoa_parameters import QAOAVariationalBaseParams
from openqaoa.devices import DeviceQiskit, create_device
from openqaoa.backends.qaoa_backend import *
from openqaoa.optimizers import get_optimizer
from qiskit import QuantumCircuit
from openqaoa.utilities import *
from openqaoa.workflows.optimizer import RQAOA, QAOA
from openqaoa.problems.problem import MinimumVertexCover, PUBO, Encoding
from openqaoa.utilities import ground_state_hamiltonian
from openqaoa.qaoa_parameters import Hamiltonian

ModuleNotFoundError: No module named 'openqaoa.backends.simulators.qaoa_me_sim'

In [ ]:
classical_terms = [(0, 1),(1, 2),(2, 3), (3, 0)]
coeffs = [1, 1, 1, 1]
constant = 0
cost_hamil = Hamiltonian.classical_hamiltonian(terms=classical_terms,coeffs=coeffs,constant=constant)
mixer_hamil = X_mixer_hamiltonian(n_qubits=4)
circuit_params = QAOACircuitParams(cost_hamil, mixer_hamil, p=1)
variate_params = create_qaoa_variational_params(circuit_params,'standard','rand')

T1=5e-5 #relaxation time constant in s
T2=2*T1 #dephasing time constant in s

t_gate_list = list([20e-9, 200e-9]) #1-qubit and 2-qubit gate time in s
gate_error_list = list([12e-4, 3e-2]) #1-qubit and 2-qubit gate error probability
spam_error_prob = 5e-2 #combined SPAM error
meas1_prep0_prob = 1e-2 #classical readout error
meas0_prep1_prob = 4e-2 #classical readout error
overrot_st_dev = 2
t_readout = 5800e-9

me_sim = create_device('local','mesolver')
me_backend = get_qaoa_backend(circuit_params=circuit_params, device=me_sim, n_shots=10, noise_model={'decay': False, 'dephasing': False, 'overrot': False, 'spam': False, 'readout01': False, 'readout10': False, 'depol1': False, 'depol2': False}, times=t_gate_list + [t_readout], allowed_jump_qubits=None)

mc_sim = create_device('local','mcsolver')
mc_backend = get_qaoa_backend(circuit_params=circuit_params, device=mc_sim, n_shots=1, noise_model={'decay': 5e-5, 'dephasing': False, 'overrot': False, 'spam': False, 'readout01': False, 'readout10': False, 'depol1': False, 'depol2': False}, times=t_gate_list + [t_readout], allowed_jump_qubits=None)

In [ ]:
grid_number = 100
grid_list = list(np.linspace(start=0, stop=2*np.pi, num=grid_number, endpoint=False))
expected_cost = [[0 for x in range(grid_number)]]

gamma = np.pi/8

results_me = list()
results_mc = list()
for b,beta in enumerate(grid_list):
    variate_params = QAOAVariationalStandardParams(circuit_params, betas=[beta], gammas=[gamma])
    results_me.append(me_backend.expectation(variate_params))
    results_mc.append(mc_backend.expectation(variate_params))
    
integer_grid_list = [int(x*grid_number/(2*np.pi)) for x in grid_list]
plt.plot(integer_grid_list, results_me)
plt.plot(integer_grid_list, results_mc)
plt.show()



In [ ]:
variate_params = create_qaoa_variational_params(circuit_params,'standard','rand')
print(me_backend.get_results(variate_params))
print(me_backend.get_counts(variate_params))
print(me_backend.expectation(variate_params))
print(me_backend.probability_dict(variate_params))

print(mc_backend.get_results(variate_params))
print(mc_backend.get_counts(variate_params))
print(mc_backend.expectation(variate_params))
print(mc_backend.probability_dict(variate_params))